In [ ]:
# TRAINING ENSEMBLE MULTI-MODELS
import sys
import pandas as pd
from utils.preprocess import get_data_generators
from ai_models.ensemble_model import run_ensemble_model
sys.path.append(r'D:\skin_disease_detection\backend\app')

# Prepare data generators
train_generator, validation_generator, test_generator, label_encoder, class_weights = get_data_generators()
# Run the ensemble model
run_ensemble_model(train_generator, validation_generator, test_generator, class_weights)

In [ ]:
# TRAINING SINGLE EFFICIENTNETB3 MODEL - VERSION 1
import sys
import pandas as pd
from utils.preprocess import get_data_generators
from ai_models.efficientnet_model import train_efficientnetb3, evaluate_model
sys.path.append(r'D:\skin_disease_detection\backend\app')

# Load data generators
train_gen, val_gen, test_gen, label_enc, class_wts = get_data_generators()

# Train EfficientNetB3
model, history = train_efficientnetb3(train_gen, val_gen, class_wts)

# Evaluate on Test Set
evaluate_model(model, test_gen)


In [ ]:
# TRAINING SINGLE EFFICIENTNETB3 MODEL - VERSION 2
import sys
import pandas as pd
from utils.preprocess_2 import get_data_generators
from ai_models.efficientnet_model import train_efficientnetb3, evaluate_model
sys.path.append(r'D:\skin_disease_detection\backend\app')

# Load data generators
train_generator, validation_generator, test_generator, class_weights = get_data_generators()

# Train EfficientNetB3
model, history = train_efficientnetb3(train_generator, validation_generator, class_weights)

# Evaluate on Test Set
evaluate_model(model, test_generator)

In [ ]:
# TRAINING SINGLE EFFICIENTNETB3 MODEL - VERSION 3
import sys
import pandas as pd
from utils.preprocess_3 import get_data_generators
from ai_models.efficientnet_model import train_efficientnetb3, evaluate_model
sys.path.append(r'D:\skin_disease_detection\backend\app')

# Load data generators
train_generator, validation_generator, test_generator, class_weights = get_data_generators()

# Train EfficientNetB3
model, history = train_efficientnetb3(train_generator, validation_generator, class_weights)

# Evaluate on Test Set
evaluate_model(model, test_generator)

In [ ]:
# TRAINING SINGLE XCEPTION MODEL - VERSION 3
import sys
import pandas as pd
from utils.preprocess_3 import get_data_generators
from ai_models.xception_model import train_xception_model, evaluate_xception_model
sys.path.append(r'D:\skin_disease_detection\backend\app')

# Load data generators
train_generator, validation_generator, test_generator, class_weights = get_data_generators()

# Train xception model
model, history = train_xception_model(train_generator, validation_generator, class_weights)

# Evaluate on Test Set
evaluate_xception_model(model, test_generator)

In [ ]:
# TRAINING SINGLE CUSTOM-CNN MODEL
import sys
import pandas as pd
from utils.preprocess import get_data_generators
from ai_models.cnn_model import train_custom_cnn, evaluate_model
sys.path.append(r'D:\skin_disease_detection\backend\app')

# Load data generators
train_generator, validation_generator, test_generator, label_encoder, class_weights = get_data_generators()

# Train the model
model, history = train_custom_cnn(train_generator, validation_generator)

# Evaluate on validation set
evaluate_model(model, test_generator)

In [ ]:
# CHECKING ENSEMBLE MULTI-MODEL PREDICTIONS
import sys
import numpy as np
from tensorflow.keras.models import load_model
from joblib import load
from tensorflow.keras.preprocessing.image import load_img, img_to_array

sys.path.append(r'D:\skin_disease_detection\trained_models')

# Load base models
resnet_model = load_model(r"D:\skin_disease_detection\trained_models\ResNet50_model.h5")
print("ResNet50 model loaded successfully!")
efficientnet_model = load_model(r"D:\skin_disease_detection\trained_models\EfficientNetB0_model.h5")
print("EfficientNet model loaded successfully!") 
densenet_model = load_model(r"D:\skin_disease_detection\trained_models\DenseNet121_model.h5")
print("DenseNet model loaded successfully!")

# Load meta-model
meta_model = load(r"D:\skin_disease_detection\trained_models\meta_model.pkl")
print("Meta-model loaded successfully!")

# Load an image
image_path = r"D:\skin_disease_detection\backend\data\Ham10000\HAM10000_images_part_2\ISIC_0029360.jpg"
img = load_img(image_path, target_size=(224, 224))  # Resize as per model input size
print(img)

# Preprocess the image
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Convert single image to batch
img_array = img_array / 255.0 

# Get predictions from the base models
resnet_pred = resnet_model.predict(img_array)
efficientnet_pred = efficientnet_model.predict(img_array)
densenet_pred = densenet_model.predict(img_array)

print("ResNet50 Prediction:", resnet_pred)
print("EfficientNet Prediction:", efficientnet_pred)
print("DenseNet Prediction:", densenet_pred)

# Combine predictions for the meta-model
base_predictions = np.array([resnet_pred, efficientnet_pred, densenet_pred]).reshape(1, -1)

# Get the final prediction from the meta-model
meta_prediction = meta_model.predict(base_predictions)
print("Meta-Model Final Prediction:", meta_prediction)

In [ ]:
# CHECKING SINGLE-MODEL EFICIENTNETB3 PREDICTIONS
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image

# Load the trained model
MODEL_PATH = r'D:\skin_disease_detection\trained_models\efficientnetb3.h5'  # Update as needed
model = tf.keras.models.load_model(MODEL_PATH)

# Define class labels (ensure these match your training labels)
class_labels = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']  # Confirm correct order

def preprocess_image(image_path, target_size=(224, 224)):
    """Preprocess an image for model prediction."""
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize
    return img_array

def predict_skin_disease(image_path):
    """Predict skin disease and print detailed output."""
    img_array = preprocess_image(image_path)
    
    # Get raw predictions
    predictions = model.predict(img_array)
    
    # Print softmax probabilities for debugging
    print("\n########## SOFTMAX PROBABILITIES ##########")
    for i, prob in enumerate(predictions[0]):
        print(f"{class_labels[i]}: {prob:.4f}")

    # Get most likely class
    predicted_class = np.argmax(predictions)  
    confidence = np.max(predictions)  

    # Map to class label
    predicted_label = class_labels[predicted_class]

    print(f"\nPredicted Class: {predicted_label} (Confidence: {confidence:.2f})")

# Test with a real image
TEST_IMAGE_PATH = r'D:\skin_disease_detection\backend\own_data\base_dir\test_dir\df\ISIC_0024330.jpg' # Update as needed
predict_skin_disease(TEST_IMAGE_PATH)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input

# Optionally, if you have a mapping from indices to class names:
# Replace the dictionary below with your actual class mapping.
class_indices = {
    "akiec": 0,
    "bcc": 1,
    "bkl": 2,
    "df": 3,
    "mel": 4,
    "nv": 5,
    "vasc": 6
}
# Reverse mapping from index to class name
idx2class = {v: k for k, v in class_indices.items()}

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    """
    Loads an image from disk, converts it from BGR to RGB,
    resizes it to target_size, preprocesses it for EfficientNet,
    and expands dimensions to create a batch of size 1.
    """
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or cannot be read: " + image_path)
    # Convert BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Resize image
    image = cv2.resize(image, target_size)
    # Preprocess using EfficientNet preprocessing function
    image = preprocess_input(image)
    # Expand dimensions to match the model's input (batch_size, height, width, channels)
    image = np.expand_dims(image, axis=0)
    return image

# Load your saved model
model_path = r"D:\skin_disease_detection\trained_models\efficientnetb3.h5"  # Adjust path if needed
model = load_model(model_path)
print("Model loaded successfully.")

# Provide the path to your test image
image_path = r"D:\skin_disease_detection\backend\own_data\base_dir\test_dir\mel\ISIC_0027350.jpg"  # Replace with your image file path

# Load and preprocess the image
input_image = load_and_preprocess_image(image_path, target_size=(224, 224))

# Make a prediction
predictions = model.predict(input_image)
predicted_index = np.argmax(predictions, axis=1)[0]
predicted_class = idx2class.get(predicted_index, str(predicted_index))
print("Predicted class index:", predicted_index)
print("Predicted class name:", predicted_class)

# Display the image along with its predicted label
orig_image = cv2.imread(image_path)
orig_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
plt.imshow(orig_image)
plt.title(f"Predicted: {predicted_class}")
plt.axis("off")
plt.show()
